In [1]:
pip install gradio


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.4 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.2-py3-none-any.whl siz

In [2]:
pip install gtts


In [3]:
pip install tensorflow==2.15.0


In [4]:
pip install keras==3.0.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 7.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.


In [5]:
import cv2
import gradio as gr
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input
from gtts import gTTS
from io import BytesIO

# Define the function to preprocess a single video
def preprocess_video(video_path, target_size=(224, 224), max_frames=50):
    frames = []
    # Load video using OpenCV
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to target size and preprocess as needed
        frame = cv2.resize(frame, target_size)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        frames.append(frame)
        frame_count += 1
        if frame_count == max_frames:
            break
    cap.release()
    # Pad or truncate frames to ensure fixed length
    if len(frames) < max_frames:
        frames.extend([frames[-1]] * (max_frames - len(frames)))
    else:
        frames = frames[:max_frames]
    # Stack frames into a single video tensor
    video_tensor = np.stack(frames)
    return video_tensor

# Define the function to extract features from video frames using ResNet50
def extract_features(video_frames, base_model):
    # Preprocess input frames for ResNet50 model
    frames_preprocessed = preprocess_input(video_frames)
    # Extract features using ResNet50
    features = base_model.predict(frames_preprocessed)
    # Pool features across frames (e.g., average pooling)
    pooled_features = tf.reduce_mean(features, axis=(1, 2))  # Average pooling over spatial dimensions
    return pooled_features

# Load the saved model
saved_model_path = '/content/sign_language_rec_model.h5'
model = load_model(saved_model_path)

# Load the pre-trained ResNet50 model (excluding the top classification layer)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define the function to predict sign language from webcam frames
def predict_sign_language(frame):
    # Preprocess the frame
    video_tensor = preprocess_video(frame)
    # Extract features from the frame
    video_features = extract_features(video_tensor, base_model)
    # Reshape the features
    video_features = np.expand_dims(video_features, axis=0)
    # Perform prediction
    predictions = model.predict(video_features)
    # Get the predicted class index
    predicted_class_index = np.argmax(predictions)
    # Define a dictionary mapping class indices to words
    class_index_to_word = {
        0: 'accident', 1: 'africa', 2: 'all', 3: 'apple', 4: 'basketball', 5: 'bed', 6: 'before', 7: 'bird', 8: 'birthday',
        9: 'black', 10: 'blue', 11: 'book', 12: 'bowling', 13: 'brown', 14: 'but', 15: 'can', 16: 'candy', 17: 'chair',
        18: 'change', 19: 'cheat', 20: 'city', 21: 'clothes', 22: 'color', 23: 'cool', 24: 'cook', 25: 'cool',
        26: 'corn', 27: 'cousin', 28: 'cow', 29: 'dance', 30: 'dark', 31: 'deaf', 32: 'decide', 33: 'doctor', 34: 'dog',
        35: 'drink', 36: 'eat', 37: 'enjoy', 38: 'family', 39: 'fine', 40: 'finish', 41: 'fish', 42: 'forget', 43: 'full',
        44: 'give', 45: 'go', 46: 'graduate', 47: 'hat', 48: 'hearing', 49: 'help', 50: 'hot', 51: 'how', 52: 'jacket',
        53: 'kiss', 54: 'language', 55: 'last', 56: 'later', 57: 'letter', 58: 'like', 59: 'man', 60: 'many', 61: 'medicine',
        62: 'meet', 63: 'mother', 64: 'need', 65: 'no', 66: 'now', 67: 'orange', 68: 'paint', 69: 'paper', 70: 'pink',
        71: 'pizza', 72: 'play', 73: 'pull', 74: 'purple', 75: 'right', 76: 'same', 77: 'school', 78: 'secretary',
        79: 'shirt', 80: 'short', 81: 'son', 82: 'study', 83: 'table', 84: 'tall', 85: 'tell', 86: 'thanksgiving',
        87: 'thin', 88: 'thursday', 89: 'time', 90: 'walk', 91: 'want', 92: 'what', 93: 'white', 94: 'who',
        95: 'woman', 96: 'work', 97: 'wrong', 98: 'year', 99: 'yes'
    }
    # Get the predicted word
    predicted_word = class_index_to_word.get(predicted_class_index, 'Unknown')
    # Convert the predicted word to speech
    tts = gTTS(predicted_word)
    byte_io = BytesIO()
    tts.write_to_fp(byte_io)
    byte_io.seek(0)
    speech_output = byte_io.getvalue()
    return predicted_word, speech_output

# Create a Gradio interface
webcam_input = gr.Video(label="Please make sign language ")
predicted_word_output = gr.Label(label="Predicted Word")
speech_output = gr.Audio(label="Speech")
gr.Interface(predict_sign_language, webcam_input, [predicted_word_output, speech_output], title="Sign Language Recognition").launch()



94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://fa2e4b3889dcf9d0a9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import preprocess_input

# Define the function to preprocess a single video
def preprocess_video(video_path, target_size=(224, 224), max_frames=50):
    frames = []
    # Load video using OpenCV
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Resize frame to target size and preprocess as needed
        frame = cv2.resize(frame, target_size)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB
        frames.append(frame)
        frame_count += 1
        if frame_count == max_frames:
            break
    cap.release()
    # Pad or truncate frames to ensure fixed length
    if len(frames) < max_frames:
        frames.extend([frames[-1]] * (max_frames - len(frames)))
    else:
        frames = frames[:max_frames]
    # Stack frames into a single video tensor
    video_tensor = np.stack(frames)
    return video_tensor

# Define the function to extract features from video frames using ResNet50
def extract_features(video_frames, base_model):
    # Preprocess input frames for ResNet50 model
    frames_preprocessed = preprocess_input(video_frames)
    # Extract features using ResNet50
    features = base_model.predict(frames_preprocessed)
    # Pool features across frames (e.g., average pooling)
    pooled_features = tf.reduce_mean(features, axis=(1, 2))  # Average pooling over spatial dimensions
    return pooled_features

# Load the saved model
saved_model_path = '/content/sign_lang_rec_model.h5'
model = load_model(saved_model_path)

# Load the pre-trained ResNet50 model (excluding the top classification layer)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Define the path to your test video
test_video_path = '/content/sample.webm'

# Preprocess the test video
test_video = preprocess_video(test_video_path)

# Extract features from the test video
test_features = extract_features(test_video, base_model)

# Reshape the features to match the input shape expected by the model
test_features = np.expand_dims(test_features, axis=0)

# Perform inference using the model
predictions = model.predict(test_features)

# Get the predicted class index
predicted_class_index = np.argmax(predictions)

# Define a dictionary mapping class indices to words
class_index_to_word = {
    0: 'accident', 1: 'africa', 2: 'all', 3: 'apple', 4: 'basketball', 5: 'bed', 6: 'before', 7: 'bird', 8: 'birthday',
    9: 'black', 10: 'blue', 11: 'book', 12: 'bowling', 13: 'brown', 14: 'but', 15: 'can', 16: 'candy', 17: 'chair',
    18: 'change', 19: 'cheat', 20: 'city', 21: 'clothes', 22: 'color', 23: 'computer', 24: 'cook', 25: 'cool',
    26: 'corn', 27: 'cousin', 28: 'cow', 29: 'dance', 30: 'dark', 31: 'deaf', 32: 'decide', 33: 'doctor', 34: 'dog',
    35: 'drink', 36: 'eat', 37: 'enjoy', 38: 'family', 39: 'fine', 40: 'finish', 41: 'fish', 42: 'forget', 43: 'full',
    44: 'give', 45: 'go', 46: 'graduate', 47: 'hat', 48: 'hearing', 49: 'help', 50: 'hot', 51: 'how', 52: 'jacket',
    53: 'kiss', 54: 'language', 55: 'last', 56: 'later', 57: 'letter', 58: 'like', 59: 'man', 60: 'many', 61: 'medicine',
    62: 'meet', 63: 'mother', 64: 'need', 65: 'no', 66: 'now', 67: 'orange', 68: 'paint', 69: 'paper', 70: 'pink',
    71: 'pizza', 72: 'play', 73: 'pull', 74: 'purple', 75: 'right', 76: 'same', 77: 'school', 78: 'secretary',
    79: 'shirt', 80: 'short', 81: 'son', 82: 'study', 83: 'table', 84: 'tall', 85: 'tell', 86: 'thanksgiving',
    87: 'thin', 88: 'thursday', 89: 'time', 90: 'walk', 91: 'want', 92: 'what', 93: 'white', 94: 'who',
    95: 'woman', 96: 'work', 97: 'wrong', 98: 'year', 99: 'yes'
}

# Get the predicted word
predicted_word = class_index_to_word.get(predicted_class_index, 'Unknown')

print("Predicted word:", predicted_word)


1/2 ━━━━━━━━━━━━━━━━━━━━ 6s 7s/step

2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 5s/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 226ms/step
Predicted word: many
